## Reading/ Executing Files, Absolute/ Relative Path


#### **1. Use `__file__` + `os.path` to build robust relative paths**

This is the most **standard and robust approach**, especially in modular projects. You’ll find it used in:

```python
import os

# Always safe, script-relative path
config_path = os.path.join(os.path.dirname(__file__), '..', 'config', 'config.yaml')
```

Often wrapped in a utility method like `get_project_root()` for convenience.

```python
import os


def get_project_root() -> str:
    """
    Returns the absolute path to the root of the project directory.
    Assumes this file is located at: <project_root>/utils/path_utils.py
    """
    return os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
```


---


In the line:

```python
current_dir = os.path.dirname(os.path.abspath(__file__))
```

the call to `os.path.abspath(__file__)` is **technically redundant**, because:

* `__file__` is already an absolute path **in most execution contexts**, especially when running a script directly.
* But there are some edge cases where `__file__` can be a **relative path**, such as:

  * When using `python -m module.submodule`
  * When a module is imported dynamically or via a non-standard loader
  * In certain testing frameworks (e.g. pytest, if `__file__` is monkey-patched or passed relative)

So the pattern:

```python
os.path.dirname(os.path.abspath(__file__))
```

is simply a **defensive programming convention** to guarantee that:

* You're working with an **absolute, normalized path**, regardless of how the script was executed.

---



| Pattern                                      | Works? | Needed? | Why used?                             |
| -------------------------------------------- | ------ | ------- | ------------------------------------- |
| `os.path.dirname(__file__)`                  | ✅      | Usually | Fine in most normal runs              |
| `os.path.dirname(os.path.abspath(__file__))` | ✅      | Safest  | Guarantees absolute path in all cases |

So you can simplify if you're confident in your execution context:

```python
current_dir = os.path.dirname(__file__)  # usually fine
```

But many developers (and most open-source code) still use:

```python
current_dir = os.path.dirname(os.path.abspath(__file__))  # bulletproof
```

Just for safety.

---

`os.path.normpath()`: It turn: `A//B`, `A/B/`, `A/./B` and `A/foo/../B` all will be normalized to `A/B`. 


#### **2. Or… use `pathlib` (modern and cleaner)**

Some developers prefer `pathlib` (standard since Python 3.4+):

```python
from pathlib import Path

# Script's directory
current_dir = Path(__file__).resolve().parent

# Go to project root and access config
config_path = current_dir.parent / 'config' / 'config.yaml'
```

This approach is growing in popularity due to its readability and chainable API.

---

####  What to *Avoid* (Because it breaks)

* Relying on `open('config/config.yaml')` without making paths robust
* Using `os.getcwd()` unless they **control** the working directory (e.g., CLI apps)
* Hardcoding absolute paths (`/home/user/...`) — breaks portability

---

### Summary

| Approach                         | Common?      | Robust? | Portable? | Notes                                 |
| -------------------------------- | ------------ | ------- | --------- | ------------------------------------- |
| `__file__` + `os.path.join`      | ✅ Yes        | ✅ Yes   | ✅ Yes     | Most common in structured codebases   |
| `pathlib.Path(__file__).parent`  | ✅ Yes        | ✅ Yes   | ✅ Yes     | Cleaner, modern alternative           |
| `os.getcwd()` + relative path    | ⚠️ Sometimes | ❌ No    | ❌ No      | Works only if you control working dir |
| Hardcoded relative/absolute path | ❌ No         | ❌ No    | ❌ No      | Very fragile and discouraged          |

---
